In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from collections import Counter


In [2]:
property_df = pd.read_csv("../data/curated/property_isochrones.csv")

In [3]:
property_df.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,https://www.domain.com.au/71-newmans-rd-temple...,71 Newmans Rd Templestowe VIC 3106,1250.0,4,2,2,House,enviabl acr contain eleg tranquil life style t...,207021159,145.154614,-37.749678,207021159.0,781.0,513.0,2419.0,2163.0,15071.0,Templestowe,>50.0


In [4]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,https://www.domain.com.au/71-newmans-rd-temple...,71 Newmans Rd Templestowe VIC 3106,1250.0,4,2,2,House,enviabl acr contain eleg tranquil life style t...,207021159,145.154614,-37.749678,207021159.0,781.0,513.0,2419.0,2163.0,15071.0,Templestowe,>50.0


In [5]:
y = X['cost_text'].tolist()
type(y[0])
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1, 
       inplace=True)
X.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          "SA2_NAME21": "category"}).dtypes

l = [(elem, type(elem)) for elem in X.loc[0]]
print(l)
X.head(1)

[(4, <class 'numpy.int64'>), (2, <class 'numpy.int64'>), (2, <class 'numpy.int64'>), ('House', <class 'str'>), (781.0, <class 'numpy.float64'>), (513.0, <class 'numpy.float64'>), (2419.0, <class 'numpy.float64'>), (2163.0, <class 'numpy.float64'>), (15071.0, <class 'numpy.float64'>), ('Templestowe', <class 'str'>), ('>50.0', <class 'str'>)]


/tmp/ipykernel_18581/941513204.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1,


,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins
0,4,2,2,House,781.0,513.0,2419.0,2163.0,15071.0,Templestowe,>50.0


In [6]:
type(y[0])

float

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
from sklearn.compose import ColumnTransformer

numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

clf = MLPRegressor(max_iter=5000)
#this way we don't cheat. Read more on pipelines https://scikit-learn.org/stable/modules/compose.html
pipeline = Pipeline([('transformer', preprocessor), ('estimator', clf)]) # ('reduce_dim', PCA()),



In [ ]:
print('Cross-val standardised features acc:', np.mean(cross_val_score(pipeline, X, y, cv=10)))
# 36%... not good. might need to perform PCA to improve, and further feature selection.
# can try grid search to tune hyperparams. 

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train),y_train) # need to perform a train test split here
# need to do balanced split- i.e. one of the bootstrapping methods or something to make sure all suburbs have
# a good amount of representation. 


#print(clf.coefs_)
print('parameter shapes:',[p.shape for p in clf.coefs_])
print('num layers:', clf.n_layers_)
print('acc these params:', clf.score(preprocessor.fit_transform(X_test), y_test)) # 34%
# next steps:
# build growth models for crime, income, pop, etc by suburb 
# predict these rates in a certain year
# input it into a model along with a suburb for a "Standard" house with however many beds/parking/baths etc.
# predict. 

parameter shapes: [(8, 100), (100, 1)]
num layers: 3
acc these params: 0.34004274008874225


/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

alphas = [np.power(10.0, i) for i in range(-7, 2)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, 
                                                          stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train), y_train)
print('MLP acc without tuning:', clf.score(preprocessor.fit_transform(X_test), y_test))

hidden_sizes = [[100], [10, 10]]
#arguments of MLPClassifier and a list of values for them to search and find the best.
param_grid = {'alpha': alphas, 'hidden_layer_sizes':hidden_sizes}

# Make an appropriate scoring function

gs = GridSearchCV(estimator=clf,
                  param_grid=param_grid,
                  scoring='neg_mean_squared_error',
                  cv=3,
                  n_jobs=1, # Somehow verbose will not print if you do multithread
                  verbose=1 # More verbose = More detailed print
                  )

gs.fit(preprocessor.fit_transform(X_train), y_train)
best_params = gs.best_params_
print('best_params', best_params)

clf = MLPClassifier(max_iter=2000, **best_params)
clf.fit(preprocessor.fit_transform(X_train), y_train)
print('acc with best params:', clf.score(preprocessor.fit_transform(X_test), y_test))

MLP acc without tuning: 0.4401443392068761
Fitting 3 folds for each of 18 candidates, totalling 54 fits
best_params {'alpha': 1e-06, 'hidden_layer_sizes': [100]}


ValueError: Unknown label type: (array([550., 430., 440., ..., 220., 455., 355.]),)